#DATASCI W261: Machine Learning at Scale 

In [4]:
'''
    Name:  Megan Jasek
    Email:  meganjasek@berkeley.edu
    Class Name:  W261
    Week Number:  1
    Date:  5/16/16
'''

'\n    Name:  Megan Jasek\n    Email:  meganjasek@berkeley.edu\n    Class Name:  W261\n    Week Number:  1\n    Date:  5/16/16\n'

In [3]:
'''
HW1.1. Read through the provided control script (pNaiveBayes.sh)
   and all of its comments. When you are comfortable with their
   purpose and function, respond to the remaining homework questions below. 
   A simple cell in the notebook with a print statmement with  a "done" string will suffice here.
'''
print('Done')

Done


In [9]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW1.2

import sys
import re
count = 0
WORD_RE = re.compile(r"[\w']+")

## collect user input
filename = sys.argv[1]
findword = sys.argv[2].lower()

with open (filename, "r") as myfile:
    for line in myfile.readlines():
        # Create a list of words found in each line
        words = re.findall(WORD_RE, line.lower())
        count += words.count(findword)

print('%s %d' % (findword, count))

Overwriting mapper.py


In [11]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW1.2
import sys
total = 0

for i in range(1, len(sys.argv)):
    filename = sys.argv[i]
    with open (filename, "r") as myfile:
        for line in myfile.readlines():
            words = line.split(" ")
            findword = words[0]
            total += int(words[1])

print('%s\t%d' % (findword, total))

Overwriting reducer.py


In [2]:
!chmod +x mapper.py
!chmod +x reducer.py
!chmod a+x pNaiveBayes.sh

In [12]:
# Usage: pNaiveBayes.sh m wordlist
!./pNaiveBayes.sh 4 "assistance"

In [32]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW1.3

import sys
import re
WORD_RE = re.compile(r"[\w']+")

## collect user input
filename = sys.argv[1]
vocab = sys.argv[2].lower()

# initialize counts
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = [0, 0]

with open (filename, "r") as myfile:
    for line in myfile.readlines():
        items = line.split('\t')
        ID = items[0]
        TRUTH = int(items[1])
        # Create a list of words found in each line
        words_subject = re.findall(WORD_RE, items[2].lower())
        words_body = re.findall(WORD_RE, items[3].lower())
        words_all = words_subject + words_body
        count_docs[TRUTH] += 1
        count_words[TRUTH] += len(words_all)
        vocab_terms_in_doc = words_all.count(vocab)
        count_vocab_terms[TRUTH] += vocab_terms_in_doc 
        print('%s\t%s\t%d' % (ID, TRUTH, vocab_terms_in_doc))

# print the total counts
print('count_docs\t%d\t%d' % (count_docs[0], count_docs[1]))
print('count_words\t%d\t%d' % (count_words[0], count_words[1]))
print('count_vocab_terms\t%d\t%d' % (count_vocab_terms[0], count_vocab_terms[1]))

Overwriting mapper.py


In [33]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW1.3
import sys
import math

all_docs = {}
# Initialize the counts
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = [0, 0]

# Loop through the files
for i in range(1, len(sys.argv)):
    filename = sys.argv[i]
    with open (filename, "r") as myfile:
        for line in myfile.readlines():
            elements = line.split("\t")
            ID = elements[0]
            if ID[:6] == 'count_':
                if ID == 'count_docs':
                    count_docs[0] += int(elements[1])
                    count_docs[1] += int(elements[2])
                elif ID == 'count_words':
                    count_words[0] += int(elements[1])
                    count_words[1] += int(elements[2])
                else:
                    count_vocab_terms[0] += int(elements[1])
                    count_vocab_terms[1] += int(elements[2])                 
            else:
                # save the document in the all_docs dictionary for later use
                TRUTH = int(elements[1])
                vocab_terms_in_doc = int(elements[2])
                all_docs[ID] = [TRUTH, vocab_terms_in_doc]

total_docs = sum(count_docs)
prior = [0, 0]
condprob = [0, 0]
classes = [0, 1]

# Train MultinomialNB
for cls in classes:
    prior[cls] = count_docs[cls] / float(total_docs)
    # do not use smoothing
    condprob[cls] = count_vocab_terms[cls] / float(count_words[cls])

# Apply MultinomialNB
for key, value in all_docs.iteritems():
    score = [0, 0]
    TRUTH = value[0]
    vocab_terms_in_doc = value[1]
    for cls in classes:
        score[cls] = math.log(prior[cls])
        score[cls] += vocab_terms_in_doc * math.log(condprob[cls])
    prediction = 1 if (score[1] > score[0]) else 0
    print('%s\t%d\t%d' % (key, TRUTH, prediction))

Overwriting reducer.py


In [34]:
# Usage: pNaiveBayes.sh m wordlist
!./pNaiveBayes.sh 4 "assistance"

In [17]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW1.4

from __future__ import print_function
import sys
import re

WORD_RE = re.compile(r"[\w']+")

## collect user input
filename = sys.argv[1]
vocab = re.split(" ",sys.argv[2].lower())

# initialize counts
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = {}
for term in vocab:
    count_vocab_terms[term] = [0, 0]

with open (filename, "r") as myfile:
    for line in myfile.readlines():
        items = line.split('\t')
        ID = items[0]
        TRUTH = int(items[1])
        # Create a list of words found in each line
        words_subject = re.findall(WORD_RE, items[2].lower())
        words_body = re.findall(WORD_RE, items[3].lower())
        words_all = words_subject + words_body
        count_docs[TRUTH] += 1
        count_words[TRUTH] += len(words_all)
        print('%s\t%s' % (ID, TRUTH),end="")
        for term in vocab:
            count_term = words_all.count(term)
            count_vocab_terms[term][TRUTH] += count_term
            print('\t%s\t%d' % (term, count_term),end="")
        print()

# print the total counts
print('count_docs\t%d\t%d' % (count_docs[0], count_docs[1]))
print('count_words\t%d\t%d' % (count_words[0], count_words[1]))
for term in vocab:
    print('count_vocab_term\t%s\t%d\t%d' % (term, count_vocab_terms[term][0], count_vocab_terms[term][1]))

Overwriting mapper.py


In [18]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW1.4
import sys
import math

all_docs = {}
# Initialize the counts
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = {}
vocab = []
FIRST_FILE = True

# Loop through the files
for i in range(1, len(sys.argv)):
    filename = sys.argv[i]
    with open (filename, "r") as myfile:
        for line in myfile.readlines():
            elements = line.split("\t")
            ID = elements[0]
            if ID[:6] == 'count_':
                if ID == 'count_docs':
                    count_docs[0] += int(elements[1])
                    count_docs[1] += int(elements[2])
                elif ID == 'count_words':
                    count_words[0] += int(elements[1])
                    count_words[1] += int(elements[2])
                else:
                    term = elements[1]
                    if term not in count_vocab_terms:
                        count_vocab_terms[term] = [int(elements[2]), int(elements[3])]
                    else:
                        count_vocab_terms[term][0] += int(elements[2])
                        count_vocab_terms[term][1] += int(elements[3])
                    # if it's the first file, save the vocab
                    if FIRST_FILE:
                        vocab.append(term)
            else:
                # save the document in the all_docs dictionary for later use
                TRUTH = int(elements[1])
                terms = {}
                for j in range(2, len(elements),2):
                    term = elements[j]
                    terms[term] = int(elements[j+1])
                all_docs[ID] = [TRUTH, terms]
    FIRST_FILE = False

total_docs = sum(count_docs)
prior = [0, 0]
condprob = {}
for term in vocab:
    condprob[term] = [0, 0]
classes = [0, 1]

# Train MultinomialNB
for cls in classes:
    prior[cls] = count_docs[cls] / float(total_docs)
    # do not use smoothing
    for term in vocab:
        condprob[term][cls] = count_vocab_terms[term][cls] / float(count_words[cls])

# Apply MultinomialNB
count_correct = 0
count_total = 0
for ID, value in all_docs.iteritems():
    score = [0, 0]
    TRUTH = value[0]
    vocab_terms_in_doc = value[1]
    for cls in classes:
        score[cls] = math.log(prior[cls])
        for term, count in vocab_terms_in_doc.iteritems():
            if condprob[term][cls] > 0.0:
                score[cls] += (count * math.log(condprob[term][cls]))
    prediction = 1 if (score[1] > score[0]) else 0
    print('%s\t%d\t%d' % (ID, TRUTH, prediction))
    count_total += 1
    if TRUTH == prediction:
        count_correct += 1
print('accuracy: %f' % (count_correct / float(count_total)))

Overwriting reducer.py


In [19]:
# Usage: pNaiveBayes.sh m wordlist
!./pNaiveBayes.sh 4 "assistance valium enlargementWithATypo"

In [20]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW1.5

from __future__ import print_function
import sys
import re

WORD_RE = re.compile(r"[\w']+")

## collect user input
filename = sys.argv[1]

# initialize counts
count_docs = [0, 0]
count_words = [0, 0]
vocab = []
count_vocab_terms = {}

with open (filename, "r") as myfile:
    for line in myfile.readlines():
        items = line.split('\t')
        ID = items[0]
        TRUTH = int(items[1])
        # Create a list of words found in each line
        words_subject = re.findall(WORD_RE, items[2].lower())
        words_body = re.findall(WORD_RE, items[3].lower())
        words_all = words_subject + words_body
        count_docs[TRUTH] += 1
        count_words[TRUTH] += len(words_all)
        vocab_line = []
        for term in words_all:
            if term not in vocab:
                vocab.append(term)
            if term not in vocab_line:
                vocab_line.append(term)
            if term not in count_vocab_terms:
                count_vocab_terms[term] = [0,0]
        print('%s\t%s' % (ID, TRUTH),end="")
        for term in vocab_line:
            count_term = words_all.count(term)
            count_vocab_terms[term][TRUTH] += count_term
            print('\t%s\t%d' % (term, count_term),end="")
        print()

# print the total counts
print('count_docs\t%d\t%d' % (count_docs[0], count_docs[1]))
print('count_words\t%d\t%d' % (count_words[0], count_words[1]))
for term in vocab:
    print('count_vocab_term\t%s\t%d\t%d' % (term, count_vocab_terms[term][0], count_vocab_terms[term][1]))


Overwriting mapper.py


In [25]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW1.5
import sys
import math

all_docs = {}
# Initialize the counts
count_docs = [0, 0]
count_words = [0, 0]
count_vocab_terms = {}

# Loop through the files
for i in range(1, len(sys.argv)):
    filename = sys.argv[i]
    with open (filename, "r") as myfile:
        for line in myfile.readlines():
            elements = line.split("\t")
            ID = elements[0]
            if ID[:6] == 'count_':
                if ID == 'count_docs':
                    count_docs[0] += int(elements[1])
                    count_docs[1] += int(elements[2])
                elif ID == 'count_words':
                    count_words[0] += int(elements[1])
                    count_words[1] += int(elements[2])
                else:
                    term = elements[1]
                    if term not in count_vocab_terms:
                        count_vocab_terms[term] = [int(elements[2]), int(elements[3])]
                    else:
                        count_vocab_terms[term][0] += int(elements[2])
                        count_vocab_terms[term][1] += int(elements[3])
            else:
                # save the document in the all_docs dictionary for later use
                TRUTH = int(elements[1])
                terms = {}
                for j in range(2, len(elements),2):
                    term = elements[j]
                    terms[term] = int(elements[j+1])
                all_docs[ID] = [TRUTH, terms]

total_docs = sum(count_docs)
prior = [0, 0]
condprob = {}
for term in count_vocab_terms:
    condprob[term] = [0, 0]
classes = [0, 1]

# Train MultinomialNB
for cls in classes:
    prior[cls] = count_docs[cls] / float(total_docs)
    # do not use smoothing
    for term in count_vocab_terms:
        condprob[term][cls] = (count_vocab_terms[term][cls]+1.0) / (float(count_words[cls])+1.0)

# Apply MultinomialNB
count_correct = 0
count_total = 0
for ID, value in all_docs.iteritems():
    score = [0, 0]
    TRUTH = value[0]
    vocab_terms_in_doc = value[1]
    for cls in classes:
        score[cls] = math.log(prior[cls])
        for term, count in vocab_terms_in_doc.iteritems():
            if condprob[term][cls] > 0.0:
                score[cls] += (count * math.log(condprob[term][cls]))
    prediction = 1 if (score[1] > score[0]) else 0
    print('%s\t%d\t%d' % (ID, TRUTH, prediction))
    count_total += 1
    if TRUTH == prediction:
        count_correct += 1
print('accuracy: %f' % (count_correct / float(count_total)))

Overwriting reducer.py


In [26]:
# Usage: pNaiveBayes.sh m wordlist
!./pNaiveBayes.sh 4 "assistance"

In [13]:
%%writefile mapper.py
#!/usr/bin/python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW1.5_OLD

import sys
import re

# Takes a list of words and returns a string separated by spaces of the
# unique words in the list
def findUniqueWords(word_list):
    unique = []
    for w in word_list:
        if w not in unique:
            unique.append(w)
    return ' '.join(unique)

WORD_RE = re.compile(r"[\w']+")

## collect user input
filename = sys.argv[1]
#findwords = re.split(" ",sys.argv[2].lower())
findword = sys.argv[2].lower()

with open (filename, "r") as myfile:
    for line in myfile.readlines():
        items = line.split('\t')
        ID = items[0]
        TRUTH = items[1]
        # Create a list of words found in each line
        words_subject = re.findall(WORD_RE, items[2].lower())
        words_body = re.findall(WORD_RE, items[3].lower())
        words_all = words_subject + words_body
        count_total_words = len(words_all)
        all_text = ' '.join(words_all)
        vocab = findUniqueWords(words_all)
        print('%s\t%s\t%d\t%s\t%s' % (ID, TRUTH, count_total_words, vocab, all_text))


Overwriting mapper.py


In [3]:
%%writefile reducer.py
#!/usr/bin/python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW1.5_OLD
import sys

def updateVocab(V, word_list):
    for w in word_list:
        if w not in V:
            V.add(w)
    return V

def countTokensOfTerm(word_list, term):
    return word_list.count(term)

all_docs = {}
all_vocab = set()
count_docs = [0, 0]
count_words = [0, 0]
all_text = [[], []]

# Loop through the files once to create the aggregate counts
for i in range(1, len(sys.argv)):
    filename = sys.argv[i]
    with open (filename, "r") as myfile:
        for line in myfile.readlines():
            elements = line.split("\t")
            ID = elements[0]
            TRUTH = int(elements[1])
            count_docs[TRUTH] += 1
            count_words[TRUTH] += int(elements[2])
            vocab = elements[3].split(' ')
            all_vocab = updateVocab(all_vocab, vocab)
            text = elements[4].split(' ')
            all_text[TRUTH] += text
            # save the document for later use
            all_docs[ID] = [TRUTH, text]

total_docs = sum(count_docs)
prior = [0, 0]
condprob = [{}, {}]
classes = [0, 1]

# Train MultinomialNB
for cls in classes:
    prior[cls] = count_docs[cls] / float(total_docs)
    for term in vocab:
        count_term = countTokensOfTerm(all_text[cls], term)
        condprob[cls][term] = (count_term + 1.0) / (count_words[cls] + 1.0)

# Apply MultinomialNB
for key, value in all_docs:
    score = [0, 0]
    TRUTH = value[0]
    for cls in classes:
        score[cls] = log(prior[cls])
        text = value[1]
        for term in text:
            score[cls] += log(condprob[cls][term])
    prediction = 1 if (score[1] > score[0]) else 0
    print('%s\t%d\t%d' % (key, TRUTH, prediction))

Overwriting reducer.py


In [12]:
# Usage: pNaiveBayes.sh m wordlist
!./pNaiveBayes.sh 99 "assistance"

Traceback (most recent call last):
  File "./reducer.py", line 52, in <module>
    for key, value in all_docs:
ValueError: too many values to unpack


In [16]:
%%writefile mapper2.py
#!/usr/bin/python
import sys
import re
count = 0
WORD_RE = re.compile(r"[\w']+")
filename = sys.argv[2]
findword = sys.argv[1]
with open (filename, "r") as myfile:
    #Please insert your code
    for line in myfile.readlines():
        # Create a list of words found in each line
        words = re.findall(WORD_RE, line)
        count += words.count(findword)
print count

Overwriting mapper.py


In [2]:
!chmod a+x mapper.py

#Reduce

In [22]:
%%writefile reducer2.py
#!/usr/bin/python
import sys
sum = 0
for line in sys.stdin:
    #Please insert your code
    sum += int(line)
print sum

Overwriting reducer.py


In [4]:
!chmod a+x reducer.py

# Write script to file

In [5]:
%%writefile pGrepCount.sh
ORIGINAL_FILE=$1
FIND_WORD=$2
BLOCK_SIZE=$3
CHUNK_FILE_PREFIX=$ORIGINAL_FILE.split
SORTED_CHUNK_FILES=$CHUNK_FILE_PREFIX*.sorted
usage()
{
    echo Parallel grep
    echo usage: pGrepCount filename word chuncksize
    echo greps file file1 in $ORIGINAL_FILE and counts the number of lines
    echo Note: file1 will be split in chunks up to $ BLOCK_SIZE chunks each
    echo $FIND_WORD each chunk will be grepCounted in parallel
}
#Splitting $ORIGINAL_FILE INTO CHUNKS
split -b $BLOCK_SIZE $ORIGINAL_FILE $CHUNK_FILE_PREFIX
#DISTRIBUTE
for file in $CHUNK_FILE_PREFIX*
do
    #grep -i $FIND_WORD $file|wc -l >$file.intermediateCount &
    ./mapper.py $FIND_WORD $file >$file.intermediateCount &
done
wait
#MERGEING INTERMEDIATE COUNT CAN TAKE THE FIRST COLUMN AND TOTOL...
#numOfInstances=$(cat *.intermediateCount | cut -f 1 | paste -sd+ - |bc)
numOfInstances=$(cat *.intermediateCount | ./reducer.py)
echo "found [$numOfInstances] [$FIND_WORD] in the file [$ORIGINAL_FILE]"

Writing pGrepCount.sh


#Run the file

In [6]:
!chmod a+x pGrepCount.sh

Usage: usage: pGrepCount filename word chuncksize

In [15]:
!./pGrepCount.sh License.txt COPYRIGHT 4k

found [11] [COPYRIGHT] in the file [License.txt]
